### https://towardsdatascience.com/building-and-testing-recommender-systems-with-surprise-step-by-step-d4ba702ef80b
### https://surprise.readthedocs.io/en/stable/getting_started.html

In [1]:
#%matplotlib inline
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
#import matplotlib.pyplot as plt
import numpy as np

In [2]:
#import EDA_Library as eda

In [3]:
product_df = pd.read_csv( "/Users/svattiku/Google Drive/BDA/ProjectBDA/Retail_Data/product_data.csv", 
                         dtype={'product_code':str})

In [4]:
product_df.head(5)

,Unnamed: 0,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
0,0,MMID_2041,1981-10-18,male,NaN,452001,2016-01-19,2655,MM-INDORE-MALHAR MEGA MALL,2,51535,NONPROMO,NaN,300785147.0,BB-CB-16X20X168SWG-Suitable for ROI New,2.0,Payback
1,1,MMID_2041,1981-10-18,male,NaN,452001,2016-06-21,2655,MM-INDORE-MALHAR MEGA MALL,21,57836,NONPROMO,NaN,1000315939.0,SOO FRESH ROLL PIZZA 1PLT,40.0,Payback
2,2,MMID_2041,1981-10-18,male,NaN,452001,2016-04-07,2655,MM-INDORE-MALHAR MEGA MALL,21,43394,NONPROMO,NaN,1000314595.0,SOO FRESH PUFF PEAS & POTATO 1PLT,35.0,Payback
3,3,MMID_2041,1981-10-18,male,NaN,452001,2016-03-19,2655,MM-INDORE-MALHAR MEGA MALL,21,39335,NONPROMO,NaN,300930107.0,SOFRSH VD EGGS REGULAR 6P,72.0,Payback
4,4,MMID_2041,1981-10-18,male,NaN,452001,2016-01-19,2655,MM-INDORE-MALHAR MEGA MALL,2,51535,NONPROMO,NaN,1000010393.0,SOO FRESH BREAD BROWN 400g,15.0,Payback


In [5]:
from datetime import datetime, timedelta
def getProductDataTimePeriod(dataset_df, numOfDays):
    dataset_df['transactionDate'] = pd.to_datetime(dataset_df.transactionDate, format = '%Y-%m-%d')
    dataset_df[dataset_df['transactionDate'] > pd.to_datetime('2017-04-01', format = '%Y-%m-%d')]
    return(dataset_df[dataset_df['transactionDate'] > pd.to_datetime('2017-06-30', format = '%Y-%m-%d')- timedelta(days=numOfDays)])

In [6]:
listOfProductsToBeOmitted = ["ONION LOOSE","POTATO LOOSE","TOMATO LOOSE","CORIANDER", 
                             "CUCUMBER GREEN LOOSE", "LADYFINGER LOOSE","BOTTLE GOURD LONG","CABBAGE",
                             "CAPSICUM GREEN","Carrot English Loose", "GINGER", "LEMON LOOSE",
                             "SUGAR MEDIUM LOOSE","TATA SALT PP 1Kg","CAULIFLOWER", "FB VG CHILLI RED MIRCHI",
                             "BRINJAL BHARTA PURPLE", "METHI","RIDGE GOURD", "FB SIS NAMKEENS",
                             "BB-CB-20X25X168SWG-Suitable for ROI New","BB-CB-20X25X208SWG NEW","BB-CB-27X30X208SWG NEW",
                             "BANANA ROBUSTA RAW,BB-CB-27X30X168SWG-Suitable for ROI New","GARLIC PREMIUM","GREEN PEAS",
                             "BANANA ROBUSTA RAW","BB-CB-27X30X168SWG-Suitable for ROI New","BROCCOLI","CAPSICUM RED LOOSE",
                             "MUSHROOM BUTTON"
                            ]
       
product_filtered_df = product_df[~product_df['product_description'].isin(listOfProductsToBeOmitted)]

In [7]:
product_6months_df = getProductDataTimePeriod(product_df, 180)

In [8]:
product_6months_filtered_df = getProductDataTimePeriod(product_filtered_df, 180)

In [9]:
len(product_df.product_code.unique())

204115

In [10]:
len(product_df.product_description.unique())

202527

In [11]:
product_df['store_description'].unique()

array(['MM-INDORE-MALHAR MEGA MALL', 'MM-LUDHIANA-FEROZEPUR-MF',
       'MM-MADURAI-KOCHADAI VILLAGE', 'MM-INDORE-TREASURE ISLAND',
       'MM-AMRITSAR-TRILIUM MALL', 'MM-JAMSHEDPUR-NH-33',
       'MM-HUBLI-GOKUL ROAD-AKSHAY CEN'], dtype=object)

In [12]:
store_6months_df = product_6months_df[product_6months_df['store_description'] == 'MM-INDORE-MALHAR MEGA MALL']
store_6months_df.head(5)

,Unnamed: 0,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
182,182,MMID_2049,1990-04-04,male,NaN,999999,2017-02-04,2655,MM-INDORE-MALHAR MEGA MALL,2,79235,NONPROMO,NaN,1000460195008.0,"AW16-CSLSYN-M794-BUFLO, NAVY BLUE, 42",899.00,Payback
183,183,MMID_2049,1990-04-04,male,NaN,999999,2017-02-04,2655,MM-INDORE-MALHAR MEGA MALL,2,79235,NONPROMO,NaN,1000590966.0,CLEAN & CLEAR FW NATURAL BRIGHT BT 50ml,60.00,Payback
184,184,MMID_2049,1990-04-04,male,NaN,999999,2017-02-04,2655,MM-INDORE-MALHAR MEGA MALL,2,79235,NONPROMO,NaN,300097363.0,FAIR & LOV SKINCRM MAXFAIRNESS MEN 25g,65.00,Payback
205,205,MMID_20412,1974-12-19,male,MADHYA PRADESH,482001,2017-01-21,2655,MM-INDORE-MALHAR MEGA MALL,3,42211,0001238717,BDAZ 799 FLAT MENS FASHION ASSORTED BDJ,1000447566018.0,"BUTR-3161-COBRA-A-SF-TW-SL, 34, ADMIRAL",838.95,Payback
206,206,MMID_20412,1974-12-19,male,MADHYA PRADESH,482001,2017-01-21,2655,MM-INDORE-MALHAR MEGA MALL,3,42211,NONPROMO,NaN,1000545428001.0,AFL-HANKIES-FPP-PO3-WHITE,50.00,Payback


In [13]:
store_6months_filtered_df = product_6months_filtered_df[product_6months_filtered_df['store_description'] == 'MM-INDORE-MALHAR MEGA MALL']
store_6months_filtered_df.head(5)

,Unnamed: 0,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
182,182,MMID_2049,1990-04-04,male,NaN,999999,2017-02-04,2655,MM-INDORE-MALHAR MEGA MALL,2,79235,NONPROMO,NaN,1000460195008.0,"AW16-CSLSYN-M794-BUFLO, NAVY BLUE, 42",899.00,Payback
183,183,MMID_2049,1990-04-04,male,NaN,999999,2017-02-04,2655,MM-INDORE-MALHAR MEGA MALL,2,79235,NONPROMO,NaN,1000590966.0,CLEAN & CLEAR FW NATURAL BRIGHT BT 50ml,60.00,Payback
184,184,MMID_2049,1990-04-04,male,NaN,999999,2017-02-04,2655,MM-INDORE-MALHAR MEGA MALL,2,79235,NONPROMO,NaN,300097363.0,FAIR & LOV SKINCRM MAXFAIRNESS MEN 25g,65.00,Payback
205,205,MMID_20412,1974-12-19,male,MADHYA PRADESH,482001,2017-01-21,2655,MM-INDORE-MALHAR MEGA MALL,3,42211,0001238717,BDAZ 799 FLAT MENS FASHION ASSORTED BDJ,1000447566018.0,"BUTR-3161-COBRA-A-SF-TW-SL, 34, ADMIRAL",838.95,Payback
206,206,MMID_20412,1974-12-19,male,MADHYA PRADESH,482001,2017-01-21,2655,MM-INDORE-MALHAR MEGA MALL,3,42211,NONPROMO,NaN,1000545428001.0,AFL-HANKIES-FPP-PO3-WHITE,50.00,Payback


In [14]:
# Remove the list of obvious items
 
listOfProductsToBeOmitted = ["ONION LOOSE","POTATO LOOSE","TOMATO LOOSE","CORIANDER", 
                             "CUCUMBER GREEN LOOSE", "LADYFINGER LOOSE","BOTTLE GOURD LONG","CABBAGE",
                             "CAPSICUM GREEN","Carrot English Loose", "GINGER", "LEMON LOOSE",
                             "SUGAR MEDIUM LOOSE","TATA SALT PP 1Kg","CAULIFLOWER", "FB VG CHILLI RED MIRCHI",
                             "BRINJAL BHARTA PURPLE", "METHI","RIDGE GOURD", "FB SIS NAMKEENS",
                             "BB-CB-20X25X168SWG-Suitable for ROI New","BB-CB-20X25X208SWG NEW","BB-CB-27X30X208SWG NEW",
                             "BANANA ROBUSTA RAW,BB-CB-27X30X168SWG-Suitable for ROI New","GARLIC PREMIUM","GREEN PEAS",
                             "BANANA ROBUSTA RAW","BB-CB-27X30X168SWG-Suitable for ROI New","BROCCOLI","CAPSICUM RED LOOSE",
                             "MUSHROOM BUTTON"
                            ]
product_filtered_6months_df = store_6months_df[~store_6months_df['product_description'].isin(listOfProductsToBeOmitted)]
store_filtered_6months_df = product_filtered_6months_df


In [15]:
store_cust_product_6months_df = store_6months_df.groupby(['customerID','product_description']).size().sort_values(ascending=False).reset_index(name='Rating')
store_cust_product_6months_df.head(5)

,customerID,product_description,Rating
0,MMID_20453330,DAMMANI SOYABEAN OIL JR 15L,111
1,MMID_20472560,APPLE WASHINGTON LOOSE,55
2,MMID_20470588,BB-CB-27X30X208SWG NEW,51
3,MMID_20470588,BB-CB-20X25X208SWG NEW,49
4,MMID_20471429,SOO FRESH BUNS BURGER 4P,42


In [16]:
from surprise import SVD,SVDpp,SlopeOne,NMF,NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

In [17]:
reader = Reader(rating_scale=(0, 9))

In [18]:
data_6months = Dataset.load_from_df(store_cust_product_6months_df[['customerID', 'product_description', 'Rating']], reader)

In [19]:
def EvaluateDifferentAlgorithms():
    benchmark = []
    # Iterate over all algorithms
    for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
        # Perform cross validation
        results = cross_validate(algorithm, data_6months, measures=['RMSE'], cv=3, verbose=False)
    
        # Get results & append algorithm name
        tmp = pd.DataFrame.from_dict(results).mean(axis=0)
        tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
        benchmark.append(tmp)
    
        print(pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse'))

# Uncomment if you want to execute this function        
#EvaluateDifferentAlgorithms()       

### SVD gave least RMSE value in the above test

In [20]:
#print('Using SVD')
#algo = SVD()
#cross_validate(algo, data_6months, measures=['RMSE'], cv=3, verbose=False)

In [21]:
def get_Iu_6months(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset_6months.ur[trainset_6months.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui_6months(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset_6months.ir[trainset_6months.to_inner_iid(iid)])
    except ValueError:
        return 0

In [22]:
def topRecommendedProducts6months(customerID, number):
    all_predictions_df = all_predictions_6months_df
    topN_df = all_predictions_df[all_predictions_df['uid']==customerID][['iid','err']].head(number)
    productList = ""
    n=1
    for record in topN_df.values.tolist():
        productList = productList+"\n<tr>\n<td>"+str(n)+"</td>\n" + \
                                       "<td>"+record[0]+"</td>\n"+ \
                                       "<td>"+str(round(record[1],2))+"</td>\n"+\
                                  "</tr>"
        n = n+1
        
    content = """<html>
                 <head>
                 <style>
                 table, th, td {
                 border: 1px solid black;
                 }
                 </style></head>
                 <body>
                 <h1>Recommender System</h>
                 <h2>"""+"Top products recommended for the customer with ID "+str(customerID)+" are: """"</h2>
                 <table style="width:40%">
                 <tr>
                 <th>S.No</th>
                 <th>Product</th> 
                 <th>Err</th>
                 </tr>
                 """+productList+"""
                 </table>
                 </body>
                 </html>"""
    #print(content)
    return(content)

In [23]:
trainset_6months, testset_6months = train_test_split(data_6months, test_size=0.2)

In [24]:
print("Applying SVD on training set")
algo = SVD()
predictions_6months = algo.fit(trainset_6months).test(testset_6months)
accuracy.rmse(predictions_6months)

Applying SVD on training set
RMSE: 0.5354


0.5354062770194868

In [25]:
predictions_6months

[Prediction(uid='MMID_20485066', iid='DGSH-0171-190O-A-KS-SU, 9-10, ORANGE', r_ui=1.0, est=1.0358882260338467, details={'was_impossible': False}),
 Prediction(uid='MMID_2048131', iid='VEG AFFAIRE FROZEN GREEN PEAS PP 500g', r_ui=1.0, est=1.135612438342985, details={'was_impossible': False}),
 Prediction(uid='MMID_2049888', iid='SW Kids Glass without cover Avengers', r_ui=1.0, est=1.0860080980395903, details={'was_impossible': False}),
 Prediction(uid='MMID_2041933', iid='AMUL CHEESE SLICE 100G', r_ui=1.0, est=1.1101006048473145, details={'was_impossible': False}),
 Prediction(uid='MMID_20421164', iid='EVEREST READYMIX CHAT BX 100g', r_ui=1.0, est=0.8604754689306251, details={'was_impossible': False}),
 Prediction(uid='MMID_20493381', iid='BRITANNIA BISC DIGESTIVE 100g', r_ui=1.0, est=1.0897115166641171, details={'was_impossible': False}),
 Prediction(uid='MMID_20464927', iid='GANGWAL OTHER MIX DOSA 500g', r_ui=1.0, est=1.0926532795508663, details={'was_impossible': False}),
 Prediction

In [26]:
df_6months = pd.DataFrame(predictions_6months, columns=['uid', 'iid', 'rui', 'est', 'details'])
df_6months['Iu'] = df_6months.uid.apply(get_Iu_6months)
df_6months['Ui'] = df_6months.iid.apply(get_Ui_6months)
df_6months['err'] = abs(df_6months.est - df_6months.rui)
best_predictions_6months = df_6months.sort_values(by='err')[:10]
worst_predictions_6months = df_6months.sort_values(by='err')[-10:]

In [27]:
best_predictions_6months

,uid,iid,rui,est,details,Iu,Ui,err
85529,MMID_20487249,SURF DET POWDER EXCEL EASY WASH 1.5Kg,1.0,1.000002,{'was_impossible': False},60,227,0.000002
18663,MMID_20491019,PARLE BIS GLUCOSE 140g,1.0,1.000004,{'was_impossible': False},23,243,0.000004
21905,MMID_20478865,CHANA KABULI STD LOOSE,1.0,1.000010,{'was_impossible': False},46,404,0.000010
53289,MMID_20420945,"BARE-CLASSIC-SHORTS-CS, XL, GREY MELANGE",1.0,0.999982,{'was_impossible': False},8,2,0.000018
3259,MMID_20449167,"LPY-0010-PDP-304-SH, L, BLACK",1.0,0.999981,{'was_impossible': False},44,2,0.000019
23267,MMID_20463541,HIMALAYA SHAMPOO GENTLE DAILY CARE 400m,1.0,0.999981,{'was_impossible': False},47,12,0.000019
53628,MMID_20463243,LINC GLYCER BP ASS 5,1.0,0.999978,{'was_impossible': False},42,22,0.000022
78960,MMID_20433361,"LECB-0010-01-A-45-KC-PL-AL, XL, ROYAL",1.0,0.999971,{'was_impossible': False},18,15,0.000029
75453,MMID_20482221,RED LABEL CTC TEA REG CT 500g,1.0,0.999968,{'was_impossible': False},21,127,0.000032
38280,MMID_20489091,FB SIS SWEETS,1.0,0.999966,{'was_impossible': False},45,276,0.000034


In [28]:
all_predictions_6months_df = df_6months.sort_values(by='err')

In [29]:
#topRecommendedProducts6months('MMID_20453330', 10, all_predictions_6months_df)

In [36]:
topRecommendedProducts6months('MMID_20487249', 10)

'<html>\n                 <head>\n                 <style>\n                 table, th, td {\n                 border: 1px solid black;\n                 }\n                 </style></head>\n                 <body>\n                 <h1>Recommender System</h>\n                 <h2>Top products recommended for the customer with ID MMID_20487249 are: </h2>\n                 <table style="width:40%">\n                 <tr>\n                 <th>S.No</th>\n                 <th>Product</th> \n                 <th>Err</th>\n                 </tr>\n                 \n<tr>\n<td>1</td>\n<td>SURF DET POWDER EXCEL EASY WASH 1.5Kg</td>\n<td>0.0</td>\n</tr>\n<tr>\n<td>2</td>\n<td>GH SALT BLACK 100gm</td>\n<td>0.01</td>\n</tr>\n<tr>\n<td>3</td>\n<td>AMUL BUTTER 100G</td>\n<td>0.02</td>\n</tr>\n<tr>\n<td>4</td>\n<td>BB-CB-20X25X208SWG NEW</td>\n<td>0.02</td>\n</tr>\n<tr>\n<td>5</td>\n<td>CATCH SENDHA SALT BT 100g</td>\n<td>0.02</td>\n</tr>\n<tr>\n<td>6</td>\n<td>PATANJALI KESH KANTI HAIR OIL 120ml</t

In [31]:
topRecommendedProducts6months('MMID_20452739', 10)

'<html>\n                 <head>\n                 <style>\n                 table, th, td {\n                 border: 1px solid black;\n                 }\n                 </style></head>\n                 <body>\n                 <h1>Recommender System</h>\n                 <h2>Top products recommended for the customer with ID MMID_20452739 are: </h2>\n                 <table style="width:40%">\n                 <tr>\n                 <th>S.No</th>\n                 <th>Product</th> \n                 <th>Err</th>\n                 </tr>\n                 \n<tr>\n<td>1</td>\n<td>SOFRSH VD BREAD WHITE 400g</td>\n<td>0.0</td>\n</tr>\n<tr>\n<td>2</td>\n<td>MADHUR CRYSTAL SUGAR PP 5Kg</td>\n<td>0.07</td>\n</tr>\n<tr>\n<td>3</td>\n<td>SCF-0010-LG120-PR-RY-SK, 50x180, BLUE</td>\n<td>0.21</td>\n</tr>\n<tr>\n<td>4</td>\n<td>CLOSE UP TPASTE RED 80G</td>\n<td>0.24</td>\n</tr>\n<tr>\n<td>5</td>\n<td>FORTUNE SUNFLOWER OIL PP 1L</td>\n<td>0.24</td>\n</tr>\n<tr>\n<td>6</td>\n<td>KURKURE SOLID MAS

In [32]:
#topRecommendedProducts('MMID_20490135', 10) # After removing obvious items

In [33]:
#topRecommendedProducts('MMID_20490135', 10) # Before removing obvious items

In [34]:
#store_cust_product_50_df[store_cust_product_50_df['customerID'] == 'MMID_20490135'].query('product_description =="LIRIL SOAP LEMON 125G"')

In [35]:
#store_cust_product_50_df[store_cust_product_50_df['customerID'] == 'MMID_20490135']